In [ ]:
import nglview
import ipywidgets

pdb_code = "2rgp"
#pdb_code = "4i23"
#pdb_code = "4i1z"
ligand_remove = "HYZ"
#ligand_remove = "1C9"
#ligand_code = "AQ4"
compare_id = "4hjo"
api_id = "pdbe"
#mgltools_path = "/anaconda3/pkgs/mgltools-1.5.7-0"

In [ ]:
from biobb_io.api.pdb import pdb

In [ ]:
download_pdb = "download.pdb"
prop = {
  "pdb_code": pdb_code,
  "filter": ["ATOM", "HETATM"],
  "api_id": api_id
}

pdb(output_pdb_path=download_pdb,
    properties=prop)

In [ ]:
view = nglview.show_structure_file(download_pdb, default=True)

view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

In [ ]:
from biobb_structure_utils.utils.remove_ligand import remove_ligand

In [ ]:
pdb_no_ligand = "pdb_no_ligand.pdb"
prop = {
    "ligand": ligand_remove
}

remove_ligand(input_structure_path=download_pdb,
             output_structure_path = pdb_no_ligand,
            properties=prop)

In [ ]:
view = nglview.show_structure_file(pdb_no_ligand, default=False)
view.add_representation(repr_type='cartoon', 
                        selection='not het',
                       colorScheme = 'atomindex')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

In [ ]:
from biobb_structure_utils.utils.extract_chain import extract_chain

In [ ]:
pdb_single_chain = "extract_chain.pdb"
prop = {
    "chains": ["A"]
}

extract_chain(input_structure_path=pdb_no_ligand,
             output_structure_path = pdb_single_chain,
            properties=prop)

In [ ]:
view = nglview.show_structure_file(pdb_single_chain, default=False)
view.add_representation(repr_type='cartoon', 
                        selection='not het',
                        colorScheme = 'atomindex')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

In [ ]:
from biobb_io.api.pdb_cluster_zip import pdb_cluster_zip

In [ ]:
pdb_cluster = "pdb_cluster.zip"
prop = {
    "pdb_code": pdb_code,
    "filter": ["ATOM", "HETATM"],
    "cluster": 90,
    "api_id": api_id
}

pdb_cluster_zip(output_pdb_zip_path = pdb_cluster,
            properties=prop)

In [ ]:
from biobb_vs.utils.bindingsite import bindingsite

In [ ]:
output_bindingsite = "bindingsite.pdb"
prop = {
    #"ligand": ligand_code,
    "matrix_name": "blosum62",
    "gap_open": -10.0,
    "gap_extend": -0.5,
    "max_num_ligands": 15,
    "radius": 5
}

bindingsite(input_pdb_path = pdb_single_chain,
            input_clusters_zip = pdb_cluster,
            output_pdb_path = output_bindingsite,
            properties=prop)

In [ ]:
view = nglview.show_structure_file(output_bindingsite, default=False)
view.add_representation(repr_type='licorice', 
                        selection='*')
view.center()
view

In [ ]:
view = nglview.show_structure_file(pdb_single_chain, default=False)
view[0].add_representation(repr_type='cartoon', 
                        selection='not het',
                          opacity=.2,
                          color='#cccccc')

view.add_component(output_bindingsite, default=False)
view[1].add_representation(repr_type='surface', 
                           selection='*', 
                           opacity = .3,
                           radius='1.5',
                           lowResolution= True,
                           # 0: low resolution 
                           smooth=1,
                           useWorker= True,
                           wrap= True)
view[1].add_representation(repr_type='licorice', 
                        selection='*')

view[0].center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

In [ ]:
from biobb_vs.utils.box import box

In [ ]:
output_box = "box.pdb"
prop = {
    "offset": 2,
    "box_coordinates": True
}

box(input_pdb_path = output_bindingsite,
            output_pdb_path = output_box,
            properties=prop)

In [ ]:
#view = nglview.show_structure_file(box, default=False)
view = nglview.NGLWidget()
#s = view.add_component(pdb_single_chain)
s = view.add_component(download_pdb)
b = view.add_component(output_box)
s = view.add_component(output_bindingsite)

atomPair = [
    [ "9999:Z.ZN1", "9999:Z.ZN2" ],
    [ "9999:Z.ZN2", "9999:Z.ZN4" ],
    [ "9999:Z.ZN4", "9999:Z.ZN3" ],
    [ "9999:Z.ZN3", "9999:Z.ZN1" ],
    
    [ "9999:Z.ZN5", "9999:Z.ZN6" ],
    [ "9999:Z.ZN6", "9999:Z.ZN8" ],
    [ "9999:Z.ZN8", "9999:Z.ZN7" ],
    [ "9999:Z.ZN7", "9999:Z.ZN5" ],
    
    [ "9999:Z.ZN1", "9999:Z.ZN5" ],
    [ "9999:Z.ZN2", "9999:Z.ZN6" ],
    [ "9999:Z.ZN3", "9999:Z.ZN7" ],
    [ "9999:Z.ZN4", "9999:Z.ZN8" ]
]

#view.shape.add_cylinder( [ 0, 2, 7 ], [ 10, 0, 9 ], [ 1, 0, 0 ], 0.1 )

# structure
s.add_representation(repr_type='cartoon', 
                        selection='not het',
                        color='#cccccc',
                       opacity=.2)
# ligands box
b.add_representation(repr_type='ball+stick', 
                        selection='9999',
                        color='pink',
                       aspectRatio = 10)
# lines box
b.add_representation(repr_type='distance', 
                        atomPair= atomPair,
                       labelColor= 'transparent',
                       color= 'black')

# output bindingsite
s.add_representation(repr_type='surface', 
                        selection='*', 
                        color='skyblue',
                        lowResolution= True,
                        # 0: low resolution 
                        smooth=1,
                        surfaceType= 'av', 
                        contour=True,
                        opacity=0.4,
                        useWorker= True,
                        wrap= True)


view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

In [ ]:
from biobb_io.api.ideal_sdf import ideal_sdf

In [ ]:
sdf_ideal = "ideal.sdf"
prop = {
  "ligand_code": ligand_remove
}

ideal_sdf(output_sdf_path=sdf_ideal,
    properties=prop)

In [ ]:
from biobb_chemistry.babelm.babel_convert import babel_convert

In [ ]:
ligand = "ligand.pdb"
prop = {
    "input_format": "sdf",
    "output_format": "pdb",
    "obabel_path": "obabel"
}

babel_convert(input_path = sdf_ideal,
            output_path = ligand,
            properties=prop)

In [ ]:
# ONLY IN CASE NOT IDEAL SDF DOWNLOADED
from biobb_structure_utils.utils.extract_heteroatoms import extract_heteroatoms

In [ ]:
# ONLY IN CASE NOT IDEAL SDF DOWNLOADED
ligand = "ligand.pdb"
prop = {
    "heteroatoms": [{ "name": ligand_remove }]
}

extract_heteroatoms(input_structure_path = download_pdb,
            output_heteroatom_path = ligand,
            properties=prop)

In [ ]:
from biobb_chemistry.ambertools.reduce_add_hydrogens import reduce_add_hydrogens

In [ ]:
ligand_H = "ligand_H.pdb"
prop = {
    
}

reduce_add_hydrogens(input_path=ligand,
        output_path=ligand_H,
        properties=prop)

In [ ]:
from ipywidgets import HBox

In [ ]:
v0 = nglview.show_structure_file(ligand)
v1 = nglview.show_structure_file(ligand_H)

v0._set_size('500px', '')
v1._set_size('500px', '')

def on_change(change):
    v1._set_camera_orientation(change['new'])
    
v0.observe(on_change, ['_camera_orientation'])

HBox([v0, v1])

In [ ]:
from biobb_chemistry.babelm.babel_convert import babel_convert

In [ ]:
prep_ligand = "prep_ligand.pdbqt"
prop = {
    "input_format": "pdb",
    "output_format": "pdbqt",
    "obabel_path": "obabel"
}

babel_convert(input_path = ligand_H,
            output_path = prep_ligand,
            properties=prop)

In [ ]:
from biobb_structure_utils.utils.str_check_add_hydrogens import str_check_add_hydrogens

In [ ]:
prep_receptor = "prep_receptor.pdbqt"
prop = {
    "charges": True,
    "mode": "auto"
}

str_check_add_hydrogens(input_structure_path = pdb_single_chain,
                        output_structure_path = prep_receptor,
                        properties=prop)

In [ ]:
from biobb_vs.vina.autodock_vina import autodock_vina

In [ ]:
output_vina_pdbqt = "output_vina.pdbqt"
output_vina_log = "output_vina.log"

autodock_vina(input_ligand_pdbqt_path = prep_ligand,
             input_receptor_pdbqt_path = prep_receptor,
             input_box_path = output_box,
            output_pdbqt_path = output_vina_pdbqt,
             output_log_path = output_vina_log)

In [ ]:
models = 'all'
#models = '/0 or /1 or /4'

v0 = nglview.show_structure_file(output_vina_pdbqt, default=False)
v0.add_representation(repr_type='licorice', 
                        selection=models,
                       colorScheme= 'partialCharge')
v0.center()
v1 = nglview.show_structure_file(output_vina_pdbqt, default=False)
v1.add_representation(repr_type='ball+stick', 
                        selection=models)
v1.center()

v0._set_size('500px', '')
v1._set_size('500px', '')

def on_change(change):
    v1._set_camera_orientation(change['new'])
    
v0.observe(on_change, ['_camera_orientation'])

HBox([v0, v1])

In [ ]:
from Bio.PDB import PDBParser
parser = PDBParser(QUIET = True)
structure = parser.get_structure("protein", output_vina_pdbqt)
models = []
for i, m in enumerate(structure):
    models.append(('model' + str(i), i))
    
mdsel = ipywidgets.Dropdown(
    options=models,
    description='Sel. model:',
    disabled=False,
)
display(mdsel)

In [ ]:
from biobb_vs.utils.extract_model_pdbqt import extract_model_pdbqt

In [ ]:
output_pdbqt_model = "output_model.pdbqt"
prop = {
    "model": mdsel.value + 1
}

extract_model_pdbqt(input_pdbqt_path = output_vina_pdbqt,
             output_pdbqt_path = output_pdbqt_model,
            properties=prop)

In [ ]:
v0 = nglview.show_structure_file(ligand)
v1 = nglview.show_structure_file(output_pdbqt_model)

v0._set_size('500px', '')
v1._set_size('500px', '')

#def on_change(change):
#    v1._set_camera_orientation(change['new'])
    
#v0.observe(on_change, ['_camera_orientation'])

HBox([v0, v1])

In [ ]:
from biobb_chemistry.babelm.babel_convert import babel_convert

In [ ]:
output_pdb_model = "output_model.pdb"
prop = {
    "input_format": "pdbqt",
    "output_format": "pdb",
    "obabel_path": "obabel"
}

babel_convert(input_path = output_pdbqt_model,
             output_path = output_pdb_model,
            properties=prop)

In [ ]:
from biobb_structure_utils.utils.cat_pdb import cat_pdb

In [ ]:
output_structure = "output_structure.pdb"

cat_pdb(input_structure1 = pdb_single_chain,
             input_structure2 = output_pdb_model,
             output_structure_path = output_structure)

In [ ]:
# download 4hjo for comparing purposes
compare_structure = compare_id + ".pdb"
prop = {
  "pdb_code": compare_id,
  "filter": ["ATOM", "HETATM"],
  "api_id": api_id
}

pdb(output_pdb_path=compare_structure,
    properties=prop)

In [ ]:
# load reference and output
view = nglview.NGLWidget()
#v1 = view.add_component(compare_structure)
v1 = view.add_component(download_pdb)
v2 = view.add_component(output_structure)

v1.clear()
v1.add_representation(repr_type='licorice', 
                        selection='HYZ',
                      #selection='AQ4',
                     radius=0.5)

view._remote_call('setSize', target='Widget', args=['','600px'])
view

# align reference and output
code = """
var stage = this.stage;
var view = this.stage.viewer;
var clist_len = stage.compList.length;
var i = 0;
var s = [];
for(i = 0; i <= clist_len; i++){
    if(stage.compList[i] != undefined && stage.compList[i].structure != undefined) {        
       s.push(stage.compList[i])
    }
}

NGL.superpose(s[0].structure, s[1].structure, true)
s[ 0 ].updateRepresentations({ position: true })
s[ 0 ].autoView()

"""

view._execute_js_code(code)
view

In [ ]:
#v0 = nglview.show_structure_file(compare_structure)
#v0 = nglview.show_structure_file(download_pdb)

#v1 = nglview.show_structure_file(output_structure)

#v0._set_size('500px', '500px')
#v1._set_size('500px', '500px')

#def on_change(change):
#    v1._set_camera_orientation(change['new'])
    
#v0.observe(on_change, ['_camera_orientation'])

#HBox([v0, v1])